In [15]:
import tensorflow as tf

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [16]:
# Prepare the dataset
df = pd.read_csv('../data/btcusd_1-min_data.csv')
dataset = df.copy()

train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)


train_features = train_dataset.copy()
test_features = test_dataset.copy()

In [17]:
# Remove the target variable from features
train_labels = train_features.pop('Close')
test_labels = test_features.pop('Close')

In [18]:
train_dataset.describe().transpose()[['mean', 'std']]

,mean,std
Timestamp,1.539113e+09,1.233912e+08
Open,1.881222e+04,2.633597e+04
High,1.881937e+04,2.634383e+04
Low,1.880486e+04,2.632792e+04
Close,1.881222e+04,2.633594e+04
Volume,5.238907e+00,2.250611e+01


In [19]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))

In [20]:
close_ = np.array(train_features["Open"])
close_normalizer = tf.keras.layers.Normalization(input_shape=[1,], axis=None)
close_normalizer.adapt(close_)

/data/projects/personal/bitbot/forecast_app/.venv/lib/python3.12/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [21]:
close_model = tf.keras.Sequential([
    close_normalizer,
    tf.keras.layers.Dense(units=1)
])

In [22]:
close_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [ ]:
history = close_model.fit(
    train_features['Open'],
    train_labels,
    epochs=100,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)


Epoch 1/100


142465/142465 ━━━━━━━━━━━━━━━━━━━━ 279s 2ms/step - loss: 16136.1455 - val_loss: 8645.2988
Epoch 2/100
142465/142465 ━━━━━━━━━━━━━━━━━━━━ 281s 2ms/step - loss: 6289.6670 - val_loss: 9.2448
Epoch 3/100
142465/142465 ━━━━━━━━━━━━━━━━━━━━ 281s 2ms/step - loss: 9.2187 - val_loss: 9.2374
Epoch 4/100
142465/142465 ━━━━━━━━━━━━━━━━━━━━ 280s 2ms/step - loss: 9.1983 - val_loss: 9.2315
Epoch 5/100
142465/142465 ━━━━━━━━━━━━━━━━━━━━ 292s 2ms/step - loss: 9.1878 - val_loss: 9.2331
Epoch 6/100
117128/142465 ━━━━━━━━━━━━━━━━━━━━ 36s 1ms/step - loss: 9.2059

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
print(hist.tail())

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 10])
  plt.xlabel('Epoch')
  plt.ylabel('Error [MPG]')
  plt.legend()
  plt.grid(True)

In [ ]:
plot_loss(history)